<a href="https://colab.research.google.com/github/AliAqdas-repo/FallDetection/blob/main/FD_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Dependencies

In [ ]:
!pip install plotly

##Importing Dependencies

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import csv
import datetime
import matplotlib.pyplot as plt
import shutil
import plotly.express as px
import seaborn as sn

##Downloading Processed Dataset from Google Drive
We are downloading the dataset we prepared the Data Organization Notebook of this project. We have already prepared the dataset and made it available on Google Drive. You can simply download it over the Google Server to Colab

In [ ]:
!gdown --id 1YV106cRSbZu6txcPdi2iMA8xDOfe8-6b #Contains all Sensor Data contrary to what we did in Previous Notebook

In [ ]:
#Unzipping the Dataset
!unzip data_proc.zip
!mv /content/content/data_proc /content/data_proc
shutil.rmtree('/content/content')

## Data Reading Functions
We initially define some functions to be used later for reading the processed data.

In [ ]:
def list_full_path(dir):
  return [os.path.join(dir,os.path.splitext(fi)[0]) for fi in os.listdir(dir)]

In [ ]:
def read_into_array(filedir):
  #reads data into array. If file extension isn't displayed in filedir then its default to .dat
  ext=os.path.splitext(filedir)[1]
  if ext=='':
    ext='.dat'
  with open(os.path.splitext(filedir)[0]+ext,'r') as datfile :
    file_data=csv.reader(datfile,delimiter=',')
    data=list()
    for row in file_data:
      data.append([float(val) for val in row])
  return data

## Data Visualization
We are using Plotly, a library which allows us to give amazing pictorial representation of different types of data





In [ ]:
def ply_plot(datpath):
  dat=np.asarray(read_into_array(f'{datpath}.csv'))
  import plotly.graph_objects as go
  if not np.shape(dat)[1]<10:
    fig = go.Figure(data=go.Heatmap(
                      z=dat,
                      x=[r'$a_{x}$',r'$a_{y}$',r'$a_{z}$',r'$b$',r'$g_{x}$',r'$g_{y}$',r'$g_{z}$',r'$m_{x}$',r'$m_{y}$',r'$m_{z}$'],
                      hoverongaps = False,colorbar={"title":'Sensor Reading'}))

  else:
    fig = go.Figure(data=go.Heatmap(
                      z=dat,
                      x=[r'$a_{x}$',r'$a_{y}$',r'$a_{z}$',r'$g_{x}$',r'$g_{y}$',r'$g_{z}$'],
                      hoverongaps = False,colorbar={"title":'Sensor Reading'}))

  fig.update_layout(
      xaxis_title=r'$Sensor\ Labels$',
      yaxis_title=r'$Time\ Axis$',)
  fig.show()

In [ ]:
#Plotting Fall Data
datpath=sorted(list_full_path('/content/data_proc/fall_files'))[10]
ply_plot(datpath)

In [ ]:
#Plotting Activities of Daily Life (ADLs) Data
datpath=sorted(list_full_path('/content/data_proc/not_fall_files'))[0]
ply_plot(datpath)

## Data Loading

In [ ]:
#DONOT USE PANDAS. Issue with Reading Data. Reads one point less
def csv_dataloader(Path,auto_balance=False,skip_wrist=False):
  ###### Parameters #######
  # Path = Path to Data(Classes need to be split in different folders beforehand)
  # auto_balance = Balances Data by Trimming Extra Data
  # skip_wrist = Skips Samples from Wrist Measurements (as they are prone to errors)
  #########################
  classes=len(os.listdir(Path)) #number of classes
  dirs=[os.path.join(Path,folds) for folds in os.listdir(Path)]
  data=[]
  lbls=[]
  file_lbl=0
  min_len=0
  len_hist=[]
  if auto_balance:
    for dir in dirs:
      data_files=sorted(list_full_path(dir))
      if skip_wrist:
        data_files=[data_file for data_file in data_files if data_file[35]!='2']
      len_hist.append(len(data_files))
    min_len=min(len_hist)
  for dir in dirs:
    data_files=sorted(list_full_path(dir))
    if skip_wrist:
      data_files=[data_file for data_file in data_files if data_file[35]!='2']
    if auto_balance:
      data_files=data_files[:min_len-1]
    dir_data=[]
    dir_lbls=[]
    for data_file in data_files:
      file_data=np.expand_dims(np.asarray(read_into_array(f'{data_file}.csv')),2)
      dir_lbls.append(np.array(file_lbl))
      dir_data.append(file_data)
    file_lbl+=1
    data.append(np.array(dir_data))
    lbls.append(np.array(dir_lbls))
  data=np.concatenate(tuple(data),0)
  lbls=np.concatenate(tuple(lbls),0)  
  return tf.data.Dataset.from_tensor_slices((data.astype(np.float32),lbls.astype(np.uint8)))

In [ ]:
# 0=Fall 1=No Fall
loader=csv_dataloader('/content/data_proc/',False,False)

In [ ]:
#Dataset Characteristics
loader.element_spec

##Splitting Data in Train,Validation and Test

In [ ]:
def splitds(dataset,train_ratio,val_ratio,test_ratio=0,shuffle=False,shuffle_buffer=5000):
  assert train_ratio+val_ratio+test_ratio==1
  SEED=32768;
  if shuffle==True:
    dataset=dataset.shuffle(shuffle_buffer,seed=SEED)
  
  ds_size=len(dataset)
  train_ds=dataset.take(int(ds_size*train_ratio))
  val_ds=dataset.skip(int(train_ratio*ds_size)).take(int(ds_size*val_ratio))
  if not test_ratio==0:
    test_ds=dataset.skip(int((train_ratio+val_ratio)*ds_size)).take(int(ds_size*test_ratio))
    return train_ds,val_ds,test_ds

  return train_ds,val_ds

In [ ]:
train_ds,val_ds,test_ds=splitds(loader,0.8,0.1,0.1,shuffle=True,shuffle_buffer=10000)

In [ ]:
print(len(train_ds))
print(len(val_ds))
print(len(test_ds))

## Model Training

In [ ]:
model=tf.keras.Sequential(
    [tf.keras.layers.Conv2D(16,(3,2),activation='relu',input_shape=(952,10,1)),
     tf.keras.layers.MaxPool2D((2,1)),
     tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
     tf.keras.layers.MaxPool2D((2,1)),
     tf.keras.layers.Conv2D(64,(3,1),activation='relu'),
     tf.keras.layers.MaxPool2D((2,1)),
     tf.keras.layers.Conv2D(128,(3,1),activation='relu'),
     tf.keras.layers.MaxPool2D((2,1)),    
     tf.keras.layers.Conv2D(256,(3,1),activation='relu'),
     tf.keras.layers.MaxPool2D((2,1)), 
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.optimizers.Adam(0.0005),loss='binary_crossentropy',metrics=['accuracy','FalsePositives','FalseNegatives'])

In [ ]:
%load_ext tensorboard
!rm -rf ./logs/
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
lr_callback=tf.keras.callbacks.ReduceLROnPlateau('val_loss',factor=0.1,patience=2)
with tf.device('/device:GPU:0'):
  history=model.fit(train_ds.batch(1),validation_data=val_ds.batch(1),epochs=10,callbacks=[tensorboard_callback,lr_callback])

In [ ]:
plt.figure(1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Loss','Validation Loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid()
plt.show()


plt.figure(2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['Accuracy','Validation Accuracy'])
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid()
plt.show()

In [ ]:
%tensorboard --logdir logs/fit

## Model Predictions
We are running predictions on our trained model so we can quantify it accuracy on unseen data

In [ ]:
#0 = fall , 1 = No Fall
truth_lbls=[]
tpred_lbls=[]
for input,lbl in test_ds.batch(1):
  #input=tf.expand_dims(x[0],0)
  tpred_lbls.append(model.predict(input))
  truth_lbls.append(lbl)
truth_lbls=np.asarray(truth_lbls)
tpred_lbls=np.asarray(tpred_lbls)
tpred_lbls=np.squeeze(np.squeeze(tpred_lbls,1))
pred_lbls=np.where(tpred_lbls>=0.5,1,0)

In [ ]:
pred_lbls

In [ ]:
np.asarray(tf.math.confusion_matrix(truth_lbls,pred_lbls,2)).astype(np.int16)

In [ ]:
#bininopaul https://stackoverflow.com/a/35572247/13223282
cm=np.asarray(tf.math.confusion_matrix(truth_lbls,pred_lbls,2))
df_cm = pd.DataFrame(cm, index = [i for i in ['Fall','Not Fall']],
                  columns = [i for i in ['Fall','Not Fall']])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True,fmt='g')

## Loading and Training using only Two Sensors Data
Let's try using only accelerometer and gyroscope data

In [ ]:
#Accelrometer and Gyro Only
!gdown --id 1dICsvfTAV0ZDDNHVR6sYwix-NgyQDPwB 

In [ ]:
!unzip data_proc_AG.zip
!mv /content/content/data_proc_AG /content/data_proc_AG
shutil.rmtree('/content/content')

In [ ]:
#Plot Fall Data
datpath=sorted(list_full_path('/content/data_proc_AG/fall_files'))[10]
ply_plot(datpath)

In [ ]:
#Plot ADLs
datpath=sorted(list_full_path('/content/data_proc/not_fall_files'))[0]
ply_plot(datpath)

In [ ]:
# 0=Fall 1=No Fall
loader=csv_dataloader('/content/data_proc_AG/',False,False)

In [ ]:
loader.element_spec

In [ ]:
train_ds,val_ds,test_ds=splitds(loader,0.8,0.1,0.1,shuffle=True,shuffle_buffer=10000)

In [ ]:
print(len(train_ds))
print(len(val_ds))
print(len(test_ds))

In [ ]:
model=tf.keras.Sequential(
    [tf.keras.layers.Conv2D(8,(3,3),strides=(1,3),activation='relu',input_shape=(952,6,1)),
     tf.keras.layers.MaxPool2D((2,1)),
     tf.keras.layers.Conv2D(16,(3,2),activation='relu'),
     tf.keras.layers.MaxPool2D((2,1)),
     tf.keras.layers.Conv2D(32,(3,1),activation='relu'),
     tf.keras.layers.MaxPool2D((2,1)),
     tf.keras.layers.Conv2D(64,(3,1),activation='relu'),
     tf.keras.layers.MaxPool2D((2,1)),    
     tf.keras.layers.Conv2D(128,(3,1),activation='relu'),
     tf.keras.layers.MaxPool2D((2,1)),
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(32,activation='relu'),
     tf.keras.layers.Dense(16,activation='relu'),
     tf.keras.layers.Dense(8,activation='relu'),
     tf.keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.optimizers.Adam(0.001),loss='binary_crossentropy',metrics=['accuracy','FalsePositives','FalseNegatives'])

In [ ]:
%load_ext tensorboard
!rm -rf ./logs/
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
lr_callback=tf.keras.callbacks.ReduceLROnPlateau('val_loss',factor=0.1,patience=1)
with tf.device('/device:GPU:0'):
  history=model.fit(train_ds.batch(1),validation_data=val_ds.batch(1),epochs=10,callbacks=[tensorboard_callback,lr_callback])

In [ ]:
plt.figure(1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Loss','Validation Loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid()
plt.show()


plt.figure(2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['Accuracy','Validation Accuracy'])
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid()
plt.show()


In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
#0 = fall , 1 = No Fall
truth_lbls=[]
tpred_lbls=[]
for input,lbl in test_ds.batch(1):
  #input=tf.expand_dims(x[0],0)
  tpred_lbls.append(model.predict(input))
  truth_lbls.append(lbl)
truth_lbls=np.asarray(truth_lbls)
tpred_lbls=np.asarray(tpred_lbls)
tpred_lbls=np.squeeze(np.squeeze(tpred_lbls,1))
pred_lbls=np.where(tpred_lbls>=0.5,1,0)

In [ ]:
# Plotting the Confusion Matrix
cm=np.asarray(tf.math.confusion_matrix(truth_lbls,pred_lbls,2))
df_cm = pd.DataFrame(cm, index = [i for i in ['Fall','Not Fall']],
                  columns = [i for i in ['Fall','Not Fall']])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True,fmt='g')

Now we have trained our model on both datasets. Observe which one performs better by viewing the confusion matrix. Our model is now ready to be optimized and converted to FlatBuffer format to be used in Arduino.

#Converting to TensorFlow Lite 

Generating a Representative Dataset for Quantization

In [ ]:
def representative_ds_gen():
  fall_files=list_full_path('/content/data_proc_AG/fall_files')
  not_fall_files=list_full_path('/content/data_proc_AG/not_fall_files')
  shuffled=fall_files+not_fall_files
  for fno in range(len(shuffled)):
    data=np.expand_dims(np.expand_dims(np.asarray(read_into_array(f'{shuffled[fno]}.csv')),2),0)
    yield([data.astype(np.float32)])

Applying Quantization and converting the model to TensorFlow Lite Format

In [ ]:
converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite_no_quant=converter.convert()
converter.representative_dataset=representative_ds_gen
converter.optimizations=[tf.lite.Optimize.DEFAULT]
tflite_model=converter.convert()

In [ ]:
curr_date=datetime.datetime.now().strftime("%Y%m%d")
MODELS_DIR = '/content/models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
with open(f'/content/models/model_no_quant.tflite','wb') as f:
  f.write(tflite_no_quant)
with open(f'/content/models/model_quant.tflite','wb') as f:
  f.write(tflite_model)

In [ ]:
MODELS_DIR ='/content/models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TFLITE = MODELS_DIR + 'model_quant.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'

In [ ]:
# Install xxd if it is not available
!apt-get update && apt-get -qq install xxd
# Convert to a C source file, i.e, a TensorFlow Lite for Microcontrollers model
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}
# Update variable names
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

In [ ]:
#Displays model data in Flatbuffer Format
!cat {MODEL_TFLITE_MICRO}

## Test Quantized Model

In [ ]:
def run_tflite_model(tflite_file, test_image_indices):
  global test_images

  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]
  print('Input Details',input_details['dtype'])
  print('Output Details',output_details['dtype'])

  predictions = np.zeros((len(test_image_indices),), dtype=int)
  for i, test_image_index in enumerate(test_image_indices):
    test_image = test_images[test_image_index]
    test_label = test_labels[test_image_index]

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_image = test_image / input_scale + input_zero_point

    
    test_image = np.expand_dims(test_image, axis=0)#.astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = (output > 0.6)

  return predictions

In [ ]:
# Helper function to evaluate a TFLite model on all images
def evaluate_model(tflite_file, model_type):
  global test_images
  global test_labels

  test_image_indices = range(test_images.shape[0])
  predictions = run_tflite_model(tflite_file, test_image_indices)
  print(predictions)
  accuracy = (np.sum(test_labels== predictions) * 100) / len(test_images)
  
  cm=np.asarray(tf.math.confusion_matrix(test_labels,predictions,2))
  df_cm = pd.DataFrame(cm, index = [i for i in ['Fall','Not Fall']],
                  columns = [i for i in ['Fall','Not Fall']])
  plt.figure(figsize = (10,7))
  sn.heatmap(df_cm, annot=True,fmt='g')

  print('%s model accuracy is %.4f%% (Number of test samples=%d)' % (
      model_type, accuracy, len(test_images)))

In [ ]:
test_images=[]
test_labels=[]
for test_image,test_label in test_ds:
  test_images.append(test_image)
  test_labels.append(test_label)
test_images=np.asarray(test_images)
test_labels=np.asarray(test_labels)

In [ ]:
evaluate_model('/content/models/model_no_quant.tflite','Unquantized')

In [ ]:
evaluate_model('/content/models/model_quant.tflite','Quantized')